In [1]:
from bs4 import BeautifulSoup as bs
import ast
import os
import os.path
from datetime import datetime
from enum import Enum
import re
import pandas as pd
import numpy as np

RAW_HANSARD_DIR = 'scraped_content'

In [2]:
class ReportSection(Enum):
    WRITTEN = 'Written Answers to Questions'
    WRITTEN_NA = 'Written Answers to Questions for Oral Answer Not Answered by End of Question Time'
    ORAL = 'Oral Answers to Questions'

**for merging with mps.csv**

In [3]:
mp_df = pd.read_csv('mps.csv')
mp_df.Party = mp_df.Party.apply(ast.literal_eval)
mp_df.Parliaments = mp_df.Parliaments.apply(ast.literal_eval)
mps = dict(
    zip(mp_df.Name.apply(lambda x: x.lower()), # keys
    zip(mp_df.Name, mp_df.Party, mp_df.Parliaments))) # values

In [4]:
# for matching honorific+name in report to actual mp data.
# cannot simply remove honorific as the programmer doesn't have an exhaustive list
# of honorifics, and some are quite rare in everyday use (e.g. Inche Rahamat Bin Kenap).
def honorific_name_to_mp_data(honorific_name):
    honorific_name = honorific_name.lower()
    
    # try the easy way first (find and remove honorific)
    honorific_match = re.match(r'(mr|mrs|ms|mdm|dr|prof|assoc prof|assoc prof dr|inche)', honorific_name)
    if honorific_match:
        name = honorific_name[honorific_match.span()[1]+1:]
        if name in mps.keys():
            return mps[name]
        
    # the hard way (levenshtein)
    mp_names = list(mps.keys())
    min_levenshtein = 99999
    min_ind = -1
    for i in range(len(mp_names)):
        l_dist = levenshtein(mp_names[i], honorific_name)
        if l_dist < min_levenshtein:
            min_levenshtein = l_dist
            min_ind = i
    print(f'levenshtein matched {honorific_name} to {mp_names[min_ind]}')
    return mps[mp_names[min_ind]]


# borrowed from: https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/
# we use levenshtein as it helps to protect against typos too, like the "asked asked" in:
# https://sprs.parl.gov.sg/search/sprs3topic?reportid=oral-answer-2822
def levenshtein(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

**from html files, get pqs**

we assume that all pqs are prefaced with #. (non sprs) or # (sprs). ignore follow up qns since we are only interested in mapping mps to topics, and the follow up qns will always be from the same mp and on the same topic.

notes regarding minister titles:
* Minister for Culture, Community and Youth is the only minister title with a comma
* no questions were ever directed to minister mentor

In [5]:
cap_word = r'[A-Z][a-z]+'
cap_words = f'({cap_word})( {cap_word})*'
mccy = 'Minister for Culture, Community and Youth'
minister_for_something = f'({cap_words} )?Minister for {cap_words}( and (the )?{cap_words})?'
something_minister = f'{cap_words} Minister'
minister_regex = re.compile(f'(({mccy})|({minister_for_something})|({something_minister}))')

def pq_para_to_pq(pq_para):
    para = pq_para[0]
    asker_honorific_name, para = para.split(' asked the ', 1)
    minister_match = re.search(minister_regex, para)
    askee = para[:minister_match.span()[1]]
    question = para[minister_match.span()[1]:].strip()
    asker, asker_party, asker_parls = honorific_name_to_mp_data(asker_honorific_name)
    return [asker, asker_party, asker_parls, askee, question, *pq_para[1:]]


In [6]:
pqs = []
    
for file in os.listdir(RAW_HANSARD_DIR):
    filepath = os.path.join(RAW_HANSARD_DIR, file)
    if not os.path.isfile(filepath):
        continue
 
    # the files don't have consistent html formatting, but they fall into two major groups.
    if file[0:4] == 'sprs':
        is_sprs = True
    else:
        is_sprs = False
    
    with open(filepath, 'r') as f:
        soup = bs(f, 'html.parser')
        
    if is_sprs:
        # extract metadata from the table at the top of the page
        topic_table = soup.find('table', {'class': 'topic'})
        topic_text = topic_table.get_text()
        
        parl_no_label_span = re.match(r'Parliament No:', topic_text).span()
        topic_text = topic_text[parl_no_label_span[1]:]
        parl_no = int(re.match(r'\d+', topic_text).group())

        sit_date_label_span = re.match(r'.*Sitting Date:', topic_text).span()
        topic_text = topic_text[sit_date_label_span[1]:]
        sit_date = datetime.strptime(re.match(r'\d+-\d+-\d+', topic_text).group(), '%d-%m-%Y')
        
        # extract report section via filename
        if 'oral-answer' in file:
            section = ReportSection.ORAL
        elif 'written-answer-na' in file:
            section = ReportSection.WRITTEN_NA
        elif 'written-answer' in file:
            section = ReportSection.WRITTEN
        else:
            raise 'unidentified section (sprs)'
        section = section.value
        
        # extract paragraphs from report in the form of rows of table
        report_table = soup.find('div', {'class': 'reportTable'})
        report_table_rows = report_table.findChildren()

        # find out which rows correspond to pqs and which rows don't.
        for row in report_table_rows:
            paragraph = row.get_text()
            paragraph = paragraph.replace('\xa0', ' ')
            if not re.match(r'\d+.{8,75} asked the .+', paragraph):
                continue
            paragraph_no_num = paragraph[
                re.search('\d+', paragraph).span()[1]:]
            pqs.append(pq_para_to_pq((paragraph_no_num.strip(), sit_date, parl_no, section)))
    else:
        # extract metadata from meta elements directly
        sit_date_str = soup.find('meta', {'name': 'Sit_Date'})['content']
        parl_no_str = soup.find('meta', {'name': 'Parl_No'})['content']
        section_str = soup.find('meta', {'name': 'Sect_Name'})['content']
        
        sit_date = datetime.strptime(sit_date_str, '%Y-%m-%d')
        parl_no = int(parl_no_str)
        if 'NOT REACHED' in section_str:
            section = ReportSection.WRITTEN_NA
        elif 'ORAL' in section_str:
            section = ReportSection.ORAL
        elif 'WRITTEN' in section_str:
            section = ReportSection.WRITTEN
        else:
            raise 'unidentified section (non sprs)'
        section = section.value
        
        # extract paragraphs from table in the form of raw text (since each paragraph doesn't have its own element)
        report_table = soup.find('div', {'class': 'reportTable'})
        raw_text = report_table.get_text()
        
        # look for things like 1. 2. etc
        first_number_dot_occurrence = re.search(r'\d+\. .{8,75} asked the ', raw_text)
        
        if not first_number_dot_occurrence:
            continue

        # cut off everything before the actual body of the text
        raw_text = raw_text[first_number_dot_occurrence.span()[0]:]
        
        # formatting stuff
        raw_text = raw_text.replace('\n\n', ' ')
        raw_paras = raw_text.split('\xa0\xa0\xa0\xa0')
        
        new_pq_paras = list(
            map(lambda s: (s[re.search('\d+\.', s).span()[1]:].strip(), sit_date, parl_no, section), # remove index, add sitting date, parl no, section
                map(lambda s: re.sub('Column: \d+', '', s.replace('  ', ' ').strip()), # remove column indicators and extra spaces
                    filter(lambda para: re.match(r'\d+\. .{8,75} asked the ', para), raw_paras)))) # keep only paragraphs that are pqs.
        
        pqs += list(map(pq_para_to_pq, new_pq_paras))


levenshtein matched mr zhulkarnain abdul rahim asked to zhulkarnain abdul rahim
levenshtein matched mr murali pillai to murali pillai sc
levenshtein matched mr koo young to koo yong
levenshtein matched encik othman bin haron eusofe to othman bin haron eusofe
levenshtein matched mr koo young to koo yong
levenshtein matched mr eric cheong yuen chee to melvin yong yik chye
levenshtein matched mr koo young to koo yong
levenshtein matched mr eric cheong yuen chee to melvin yong yik chye
levenshtein matched mr p. govindaswamy to govindaswamy, p.
levenshtein matched dr s. vasoo to a.p. rajah
levenshtein matched mr eric cheong yuen chee to melvin yong yik chye
levenshtein matched inche rahamat bin kenap to rahmat bin kenap
levenshtein matched inche rahamat bin kenap to rahmat bin kenap
levenshtein matched inche rahamat bin kenap to rahmat bin kenap
levenshtein matched dr s. vasoo to a.p. rajah
levenshtein matched mrs yu-foo yee shoon to yu-foo yee shoon
levenshtein matched mr ong eng guan to p

In [7]:
pq_df = pd.DataFrame(pqs, columns=['asker_name', 'asker_party', 'asker_parliaments', 'askee', 'question', 'sitting_date', 'parliament_no', 'report_section'])
pq_df

,asker_name,asker_party,asker_parliaments,askee,question,sitting_date,parliament_no,report_section
0,Yip Hon Weng,[People's Action Party],"(14,)",Minister for Sustainability and the Environment,(a) whether the Ministry will consider to stre...,2022-04-04,14,Oral Answers to Questions
1,Leon Perera,[Workers' Party],"(13, 14)",Minister for Sustainability and the Environment,whether the Ministry will consider (i) easing ...,2022-04-04,14,Oral Answers to Questions
2,Zhulkarnain Abdul Rahim,[People's Action Party],"(14,)",Minister for Home Affairs,in light of the recent sword and knife-related...,2022-04-04,14,Oral Answers to Questions
3,Zhulkarnain Abdul Rahim,[People's Action Party],"(14,)",Minister for Home Affairs,whether there are legislative measures to help...,2022-04-04,14,Oral Answers to Questions
4,Tan Wu Meng,[People's Action Party],"(13, 14)",Minister for Education,(a) what is the approach taken by schools rega...,2022-04-04,14,Oral Answers to Questions
5,Yeo Wan Ling,[People's Action Party],"(14,)",Minister for Social and Family Development,(a) whether there has been an increase in the ...,2022-04-04,14,Oral Answers to Questions
6,Gan Thiam Poh,[People's Action Party],"(12, 13, 14)",Minister for Manpower,(a) how many CPF members currently have MediSa...,2022-04-05,14,Oral Answers to Questions
7,Gan Thiam Poh,[People's Action Party],"(12, 13, 14)",Minister for Manpower,(a) for those with MediSave contributions in a...,2022-04-05,14,Oral Answers to Questions
8,Leon Perera,[Workers' Party],"(13, 14)",Minister for Home Affairs,(a) whether the Government has any policy proh...,2022-04-05,14,Oral Answers to Questions
9,Zhulkarnain Abdul Rahim,[People's Action Party],"(14,)",Minister for National Development,of the HDB blocks that have undergone the Lift...,2022-04-05,14,Oral Answers to Questions


In [8]:
pq_df.to_csv('pqs.csv', index=False)